In [1]:
include("/home/kwat/github/Kraft.jl/src/Kraft.jl")

Main.Kraft

In [2]:
project_dir = dirname(@__DIR__)

"/home/kwat/github/omics_sample"

In [3]:
input_dir = joinpath(project_dir, "input")

"/home/kwat/github/omics_sample/input"

In [4]:
output_dir = joinpath(project_dir, "output")

"/home/kwat/github/omics_sample/output"

In [5]:
Kraft.check_program()

data_for_processing_sequence_dir = joinpath(input_dir, "data_for_processing_sequence")

if !isdir(data_for_processing_sequence_dir)

    Kraft.print_and_run_cmd(`unzip -o -d $input_dir $data_for_processing_sequence_dir.zip`)
    
end

Checking program...
`which skewer`
/home/kwat/miniconda3/bin/skewer
`which fastqc`
/home/kwat/miniconda3/bin/fastqc
`which bgzip`
/home/kwat/miniconda3/bin/bgzip
`which tabix`
/home/kwat/miniconda3/bin/tabix
`which minimap2`
/home/kwat/miniconda3/bin/minimap2
`which samtools`
/home/kwat/miniconda3/bin/samtools
`which bcftools`
/home/kwat/miniconda3/bin/bcftools
`which kallisto`
/home/kwat/miniconda3/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/home/kwat/miniconda3/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/home/kwat/miniconda3/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/home/kwat/miniconda3/envs/py2/bin/configureStrelkaSomaticWorkflow.py


In [6]:
snpeff = joinpath(
    input_dir,
    "snpEff",
    "snpEff.jar",
)

"/home/kwat/github/omics_sample/input/snpEff/snpEff.jar"

In [7]:
if !isfile(snpeff)
    
    throw("$snpeff is missing.")
    
end

In [8]:
using JSON: parse

In [9]:
project_json = parse(read(joinpath(project_dir, "project.json"), String))

Dict{String,Any} with 12 entries:
  "_soma_rna.2.fastq.gz" => "input/sample/soma_rna.2.1k.fastq.gz"
  "n_job"                => 10
  "germ_dna.2.fastq.gz"  => "/media/kwat/CarrotCake/garden/sample/738/WGC056190…
  "germ_dna.1.fastq.gz"  => "/media/kwat/CarrotCake/garden/sample/738/WGC056190…
  "_soma_rna.1.fastq.gz" => "input/sample/soma_rna.1.1k.fastq.gz"
  "gb_memory"            => 48
  "download"             => Dict{String,Any}("input"=>Any["https://s3-us-west-1…
  "_soma_dna.1.fastq.gz" => "input/sample/soma_dna.1.1k.fastq.gz"
  "_germ_dna.1.fastq.gz" => "input/sample/germ_dna.1.1k.fastq.gz"
  "_germ_dna.2.fastq.gz" => "input/sample/germ_dna.2.1k.fastq.gz"
  "dna_is_targeted"      => false
  "_soma_dna.2.fastq.gz" => "input/sample/soma_dna.2.1k.fastq.gz"

In [10]:
process_dna_arguments = (
    joinpath(data_for_processing_sequence_dir, "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz"),
    joinpath(data_for_processing_sequence_dir, "chromosome.bed.gz"),
    joinpath(data_for_processing_sequence_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    2,
    snpeff,
)

("/home/kwat/github/omics_sample/input/data_for_processing_sequence/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/home/kwat/github/omics_sample/input/data_for_processing_sequence/chromosome.bed.gz", "/home/kwat/github/omics_sample/input/data_for_processing_sequence/chrn_n.tsv", 10, 48, 2, "/home/kwat/github/omics_sample/input/snpEff/snpEff.jar")

In [ ]:
if all((in(key, keys(project_json)) for key in (
    "germ_dna.1.fastq.gz",
    "germ_dna.2.fastq.gz",
)))
    
    Kraft.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_germ_dna"),
        process_dna_arguments...,
    )
    
end

(2020-08-27T21:29:16.03) Trimming sequence ...
`skewer --threads 10 -x AGATCGGAAGAGC --compress --output /home/kwat/github/omics_sample/output/process_germ_dna/trim_sequence/germ --quiet /media/kwat/CarrotCake/garden/sample/738/WGC056190D_combined_R1.fastq.gz /media/kwat/CarrotCake/garden/sample/738/WGC056190D_combined_R2.fastq.gz`
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	AGATCGGAAGAGC
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Solexa/Illumina 1.3+/Illumina 1.5+ FASTQ (auto detected)
-- number of concurrent threads (-t):	10
Thu Aug 27 21:29:16 2020 >> started

Fri Aug 28 01:24:55 2020 >> done (14139.837s)
510566063 read pairs processed; of these:
   121971 ( 0.02%) short read pairs filtered 

Started analysis of germ-trimmed-pair1.fastq.gz
Started analysis of germ-trimmed-pair2.fastq.gz
Approx 5% complete for germ-trimmed-pair1.fastq.gz
Approx 5% complete for germ-trimmed-pair2.fastq.gz
Approx 10% complete for germ-trimmed-pair1.fastq.gz
Approx 10% complete for germ-trimmed-pair2.fastq.gz
Approx 15% complete for germ-trimmed-pair1.fastq.gz
Approx 15% complete for germ-trimmed-pair2.fastq.gz
Approx 20% complete for germ-trimmed-pair1.fastq.gz
Approx 20% complete for germ-trimmed-pair2.fastq.gz
Approx 25% complete for germ-trimmed-pair1.fastq.gz
Approx 25% complete for germ-trimmed-pair2.fastq.gz
Approx 30% complete for germ-trimmed-pair1.fastq.gz
Approx 30% complete for germ-trimmed-pair2.fastq.gz
Approx 35% complete for germ-trimmed-pair1.fastq.gz
Approx 35% complete for germ-trimmed-pair2.fastq.gz
Approx 40% complete for germ-trimmed-pair1.fastq.gz
Approx 40% complete for germ-trimmed-pair2.fastq.gz
Approx 45% complete for germ-trimmed-pair1.fastq.gz
Approx 45% complete fo

Analysis complete for germ-trimmed-pair1.fastq.gz
Analysis complete for germ-trimmed-pair2.fastq.gz
(2020-08-28T02:17:49.712) Done in 52 minutes, 53 seconds, 617 milliseconds.
(2020-08-28T02:17:49.712) Aligning sequence ...
pipeline(pipeline(pipeline(pipeline(`minimap2 -x sr -t 10 -K 2G -R '@RG\tID:Germ\tSM:Germ' -a /home/kwat/github/omics_sample/input/data_for_processing_sequence/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/kwat/github/omics_sample/output/process_germ_dna/trim_sequence/germ-trimmed-pair1.fastq.gz /home/kwat/github/omics_sample/output/process_germ_dna/trim_sequence/germ-trimmed-pair2.fastq.gz`, stdout=`samtools sort --threads 10 -m 2G -n`), stdout=`samtools fixmate --threads 10 -m - -`), stdout=`samtools sort --threads 10 -m 2G`), stdout>Base.FileRedirect("/home/kwat/github/omics_sample/output/process_germ_dna/align_sequence/germ.bam.tmp", false))


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::4.762*1.00] loaded/built the index for 195 target sequence(s)
[M::mm_mapopt_update::4.762*1.00] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 195
[M::mm_idx_stat::5.523*1.00] distinct minimizers: 100167746 (38.80% are singletons); average occurrences: 5.519; average spacing: 5.607
[M::worker_pipeline::480.430*9.71] mapped 13411698 sequences
[M::worker_pipeline::928.573*9.93] mapped 13408748 sequences
[M::worker_pipeline::1406.247*10.01] mapped 13412340 sequences
[M::worker_pipeline::1924.761*9.81] mapped 13408894 sequences
[M::worker_pipeline::2276.135*9.87] mapped 13404534 sequences
[M::worker_pipeline::2696.058*9.92] mapped 13405370 sequences
[M::worker_pipeline::3110.181*9.96] mapped 13401518 sequences
[M::worker_pipeline::3633.505*9.86] mapped 13398850 sequences
[M::worker_pipeline::3989.673*9.89] mapped 13413664 sequences
[M::worker_pipeline::4412

In [ ]:
if all((in(key, keys(project_json)) for key in (
    "germ_dna.1.fastq.gz",
    "germ_dna.2.fastq.gz",
    "soma_dna.1.fastq.gz",
    "soma_dna.2.fastq.gz",
)))
    
    Kraft.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )
    
end

In [ ]:
if all((in(key, keys(project_json)) for key in (
    "soma_rna.1.fastq.gz",
    "soma_rna.2.fastq.gz",
)))
    
    Kraft.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(data_for_processing_sequence_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )
    
end